In [31]:
import csv
import data_processing
import algorithm
from re import findall
import recipe_parser

In [17]:
with open('data/recipes.csv', 'r') as f:
    recipes = list(csv.DictReader(f))

In [ ]:
print(recipes[0])
rah = recipe_parser.parse_instructions(recipes[0])
print(rah)

In [18]:
recipes = recipes[:10000]
inv_idx = data_processing.build_inverted_index(recipes)
id_to_recipe = data_processing.build_id_to_recipe(recipes)
idf = data_processing.build_idf(inv_idx, len(recipes))
recipe_norms = data_processing.build_recipe_norms(inv_idx, idf)

In [19]:
res = algorithm.cosine_similarity("lemon dessert", inv_idx, idf, recipe_norms)

In [20]:
# Print out the top 10 recipe names
for recipe_id, score in res[:10]:
    print(id_to_recipe[recipe_id]['name'], score)

Rice Pudding 0.12546715457826013
Lemon Poppy Seed Cake 0.11975219083741352
Green Tomato Pie 0.11440767944197215
Lemon Cake 0.11337507196141429
Vinegar Pie 0.1126777024986445
Lemon Sour Cream Pie 0.11215789237072495
Lemon Sponge Pie 0.1100246145192226
Orange Marmalade 0.10999133807435493
Ricotta Cheesecake 0.10658745728313504
Lemon-Sour Cream Pie 0.10578394745474305


In [21]:
res2 = algorithm.algorithm(["ice cream", "strawberry", "frozen yogurt"], inv_idx, idf, recipe_norms)

In [22]:
# Print out the top 10 recipe names
for recipe_id, score in res2[:10]:
    print(id_to_recipe[recipe_id]['name'])

Homemade Yogurt
Chocolate Cream Puffs
strawberry filling
Fresh Strawberry Pie
Frozen Yogurt-On-A-Stick
Orange Pudding Cake
Low-Fat Strawberry Butter
Strawberry Sandesh
Low-Fat Berry Blue Frozen Dessert
Xocoalt
